In [1]:
import numpy as np 
import pandas as pd 
import os, datetime 
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from tqdm import tqdm
import random 
import time
import pywt
from keras import backend as K
from keras.layers import Concatenate
from keras.layers import Input, Dense, Lambda, Subtract, Add, Reshape
from keras.models import Model
from keras.optimizers import Adam

In [2]:
train_x = pd.read_csv('drive/MyDrive/bitTrader/train_x_df.csv') 
train_y = pd.read_csv('drive/MyDrive/bitTrader/train_y_df.csv')
test_x = pd.read_csv('drive/MyDrive/bitTrader/test_x_df.csv')
submission = pd.read_csv('drive/MyDrive/bitTrader/sample_submission.csv')

In [3]:
train_x

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.010004,1.010004,1.009612,1.010004,8.382875e+05,43160.632812,451.157288,7.326834e+05,37725.183594
1,0,1,7,1.009808,1.009808,1.009808,1.009808,1.622420e+05,8352.220703,39.231071,0.000000e+00,0.000000
2,0,2,7,1.009808,1.010200,1.009808,1.010200,1.664967e+04,857.377808,58.846603,1.664967e+04,857.377808
3,0,3,7,1.010200,1.011181,1.010200,1.011181,2.586971e+06,133310.343750,431.541779,2.189147e+06,112811.046875
4,0,4,7,1.010985,1.010985,1.010200,1.010200,1.129996e+06,58216.867188,176.539810,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
10159555,7361,1375,8,1.000668,1.001669,1.000626,1.001502,9.180907e+00,2203.059082,2.245034,6.229020e+00,1494.727417
10159556,7361,1376,8,1.001627,1.001920,1.000960,1.001294,7.963097e+00,1911.151611,2.211651,3.056139e+00,733.490601
10159557,7361,1377,8,1.001294,1.001461,1.000584,1.000668,3.849893e+00,923.610718,1.260224,2.284546e+00,548.042297
10159558,7361,1378,8,1.000709,1.000751,1.000042,1.000042,1.337402e+00,320.624756,0.826239,5.164965e-01,123.819839


In [4]:
def preprocess(df_2d):
    # convert 2d dataframe to 3d shape 
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


In [5]:
x_train = preprocess(train_x) 
y_train = preprocess(train_y) 
x_test  = preprocess(test_x) 

x_train.shape, y_train.shape, x_test.shape

((7362, 1380, 10), (7362, 120, 10), (529, 1380, 10))

we then standardize both train and test data. The mean and standard deviation are derived from the train set. 

In [6]:
mu = np.mean(np.concatenate([x_train, y_train], axis = 1)) 
std = np.std(np.concatenate([x_train, y_train], axis = 1)) 

print(mu, std)

119131.40956127715 3377136.213566099


In [7]:
x_train = (x_train - mu) / std 
y_train = (y_train - mu) / std 
x_test = (x_test - mu) / std

We create a simple supervised model that takes in the sequence of chart values and predicts two values as the output 
- the point at which the close price reaches its highest point 
- the value at the highest point 

In [8]:
y_train_max_pts = [] 
for i in tqdm(range(y_train.shape[0]), position = 0, leave = True):
  maxval = -1 
  time_idx = 0 
  for j in range(y_train.shape[1]):  
    if y_train[i,j,5] > maxval: 
      maxval = y_train[i,j,5]  
      time_idx = j   
  l = [maxval, time_idx]
  y_train_max_pts.append(np.asarray(l)) 

y_train_max_pts = np.asarray(y_train_max_pts) 

y_train_max_pts.shape

100%|██████████| 7362/7362 [00:00<00:00, 19310.66it/s]


(7362, 2)

Now, we are ready to train a model that trains on 10 features and outputs two values. Let's build the model first. 

In [9]:
# define RMSE custom loss
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [10]:
def build_model(timesteps, features): 
  inputs = Input((timesteps,features)) 
  bn = BatchNormalization()(inputs)
  lstm = Bidirectional(LSTM(256, return_sequences = True))(bn)
  lstm = Dropout(0.25)(lstm) 
  lstm = Bidirectional(LSTM(128, return_sequences = False))(lstm) 
  lstm = Dropout(0.25)(lstm) 
  dense = Dense(64, activation = 'relu', kernel_initializer = 'he_normal')(lstm) 
  dense = BatchNormalization()(dense) 
  dense = Dense(32, activation = 'relu', kernel_initializer = 'he_normal')(dense) 
  dense = BatchNormalization()(dense) 
  outputs = Dense(2, activation = 'relu')(dense) 
  model = Model(inputs=inputs, outputs=outputs) 
  model.compile(loss=rmse, optimizer = 'adam') 
  return model 

In [11]:
model = build_model(1380, 10)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1380, 10)]        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1380, 10)          40        
_________________________________________________________________
bidirectional (Bidirectional (None, 1380, 512)         546816    
_________________________________________________________________
dropout (Dropout)            (None, 1380, 512)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               656384    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448 

In [12]:
model_path = 'drive/MyDrive/bitTrader/proto_model/epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath=model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 15) 


In [13]:
history = model.fit(x_train, 
                   y_train_max_pts, 
                   batch_size = 32, 
                   epochs = 200, 
                   validation_split = 0.2,
                   callbacks = [early_stopping, checkpoint])

Epoch 1/200
185/185 [==============================] - 77s 373ms/step - loss: 47.4930 - val_loss: 45.3268

Epoch 00001: val_loss improved from inf to 45.32679, saving model to drive/MyDrive/bitTrader/proto_model/epoch_001_val_45.327.h5
Epoch 2/200
185/185 [==============================] - 67s 365ms/step - loss: 45.7924 - val_loss: 29.0226

Epoch 00002: val_loss improved from 45.32679 to 29.02263, saving model to drive/MyDrive/bitTrader/proto_model/epoch_002_val_29.023.h5
Epoch 3/200
185/185 [==============================] - 68s 366ms/step - loss: 42.2339 - val_loss: 27.8618

Epoch 00003: val_loss improved from 29.02263 to 27.86177, saving model to drive/MyDrive/bitTrader/proto_model/epoch_003_val_27.862.h5
Epoch 4/200
185/185 [==============================] - 67s 363ms/step - loss: 37.1774 - val_loss: 27.7490

Epoch 00004: val_loss improved from 27.86177 to 27.74898, saving model to drive/MyDrive/bitTrader/proto_model/epoch_004_val_27.749.h5
Epoch 5/200
185/185 [====================

KeyboardInterrupt: ignored

We created a model that predicts the sell time as well as the sell price. 
i.e. the model returns (sell price, sell time) 

our trading algorithm is very simple. If the predicted sell price is higher than the buy price (we are buying at the 23rd hour or at 1380th minute), our buy quantity ratio will be 1, and the sell time would be the predicted sell time.  

In [14]:
# let's check the format of the sample submission file. 
submission.head() 

,sample_id,buy_quantity,sell_time
0,0,0.480752,39
1,1,0.649667,0
2,2,0.057547,58
3,3,0.607298,10
4,4,0.461313,47


In [20]:
best_model = load_model('./drive/MyDrive/bitTrader/proto_model/epoch_016_val_27.287.h5', 
                        custom_objects = {'rmse':rmse}) 

predicted = best_model.predict(x_test) 

predicted.shape

(529, 2)

In [21]:
# scale back 
predicted[:,0] = (predicted[:,0] * std) + mu 

In [24]:
buy_prices = x_test[:,1379,5] 
buy_prices.shape # print shape of buy price array.  

(529,)

In [26]:
buy_quantity = [] 
sell_time = [] 
for i in range(buy_prices.shape[0]):
  if buy_prices[i] < predicted[i,0]:  
    buy_quantity.append(1) 
    sell_time.append(predicted[i,1])
  else: 
    buy_quantity.append(0) 
    sell_time.append(predicted[i][2]) 
     

In [34]:
buy_quantity = np.asarray(buy_quantity) 
sell_time = np.asarray(sell_time).astype(np.int32)

In [35]:
buy_quantity.shape, sell_time.shape

((529,), (529,))

In [36]:
submission.iloc[:,1] = buy_quantity 
submission.iloc[:,2] = sell_time

In [37]:
submission.head()

,sample_id,buy_quantity,sell_time
0,0,1,56
1,1,1,57
2,2,1,57
3,3,1,57
4,4,1,59


In [38]:
submission.to_csv('drive/MyDrive/bitTrader/LSTM_test.csv',index=False)